In [ ]:
import psi4
import psiresp
from rdkit import Chem
from rdkit.Chem import AllChem
import os

In [ ]:
smiles_string = "O=C(NC)[C@@H](NC(C)=O)CSC1C(N(CCC2=C(C)[N+](CC3=CC=CC=C3)=CS2)C(C1)=O)=O"

In [ ]:
mol = Chem.MolFromSmiles(smiles_string)
mol = Chem.AddHs(mol)
mol

In [ ]:
for i, atom in enumerate(mol.GetAtoms()):
    atom.SetProp("molAtomMapNumber", str(atom.GetIdx()))

mol

In [ ]:
capping_list = [2, 3, 33, 34, 35, 36, 6, 7, 8, 39, 40, 41]
backbone_list = [4, 1, 0, 5, 38]
sidechain_list = [9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 42, 43, 44,
 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 37]

In [ ]:
n_confs = 1

confs = AllChem.EmbedMultipleConfs(mol, numConfs=n_confs, randomSeed=-1)
AllChem.AlignMolConformers(mol)
AllChem.MMFFOptimizeMoleculeConfs(mol, maxIters=1000)

In [ ]:
psirespmol = psiresp.Molecule.from_rdkit(mol)
psirespmol

In [ ]:
psirespmol.optimize_geometry=True

constraints = psiresp.ChargeConstraintOptions(symmetric_atoms_are_equivalent=True)
constraints.add_charge_sum_constraint_for_molecule(psirespmol, charge=0, indices=capping_list)
constraints.add_charge_sum_constraint_for_molecule(psirespmol, charge=+1, indices=backbone_list+capping_list+sidechain_list)

In [ ]:
geometry_options = psiresp.QMGeometryOptimizationOptions(method="hf", basis="6-31g*")

esp_options = psiresp.QMEnergyOptions(method="hf", basis="6-31g*")

In [ ]:
job_resp = psiresp.Job(molecules=[psirespmol],
            charge_constraints=constraints,
            qm_optimization_options=geometry_options,
            qm_esp_options=esp_options,
            working_directory="resp_charges_generation")

In [ ]:
job_resp.generate_conformers()
job_resp.generate_orientations()
job_resp.run(update_molecules=True)

In [ ]:
os.chdir("resp_charges_generation/optimization/")
os.system("bash run_optimization.sh")
os.chdir("../../")
job_resp.run()

In [ ]:
os.chdir("resp_charges_generation/single_point/")
os.system("bash run_single_point.sh")
os.chdir("../../")
job_resp.run()

In [ ]:
molecule = job_resp.molecules[0]
molecule.charges